# Mocap Player

## Imports

## Configuration

In [1]:
import os, sys, time, subprocess

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

from PyQt5 import QtWidgets
from PyQt5.QtCore import Qt
import pyqtgraph as pg
import pyqtgraph.opengl as gl
from pathlib import Path

import motion_player 
import motion_sender
import motion_control
import motion_gui

In [3]:
# Configure mocap data

mocap_file_label_gui = widgets.Label(value="Select Mocap File", layout=widgets.Layout(width='100px'))
mocap_file_gui = FileChooser(
    button_text='Select Mocap File'
)
mocap_file_gui.default_path = 'data/mocap/'
mocap_file_gui.default_filename = 'Muriel_Take1.fbx'
mocap_file_gui.reset()

mocap_fps_label_gui = widgets.Label(value="Set Mocap FPS", layout=widgets.Layout(width='100px'))
mocap_fps_gui = widgets.IntText(value=50, layout=widgets.Layout(width='50px'))

osc_file_gui = widgets.HBox([mocap_file_label_gui, mocap_file_gui, mocap_fps_label_gui, mocap_fps_gui])
display(osc_file_gui)

# Configure OSC Receiver (for controlling the motion player)

osc_rec_ip_label_gui = widgets.Label(value="OSC Receive IP", layout=widgets.Layout(width='100px'))
osc_rec_ip0_gui = widgets.IntText(value=0, layout=widgets.Layout(width='50px'))
osc_rec_ip1_gui = widgets.IntText(value=0, layout=widgets.Layout(width='50px'))
osc_rec_ip2_gui = widgets.IntText(value=0, layout=widgets.Layout(width='50px'))
osc_rec_ip3_gui = widgets.IntText(value=0, layout=widgets.Layout(width='50px'))
osc_rec_port_label_gui = widgets.Label(value="Port", layout=widgets.Layout(width='50px'))
osc_rec_port_gui = widgets.IntText(value=9002, layout=widgets.Layout(width='100px'))

osc_rec_gui = widgets.HBox([osc_rec_ip_label_gui, osc_rec_ip0_gui, osc_rec_ip1_gui, osc_rec_ip2_gui, osc_rec_ip3_gui, osc_rec_port_label_gui, osc_rec_port_gui])
display(osc_rec_gui)

# Configure OSC Sender (for sending motion data to another application)

osc_send_ip_label_gui = widgets.Label(value="OSC Send IP", layout=widgets.Layout(width='100px'))
osc_send_ip0_gui = widgets.IntText(value=127, layout=widgets.Layout(width='50px'))
osc_send_ip1_gui = widgets.IntText(value=0, layout=widgets.Layout(width='50px'))
osc_send_ip2_gui = widgets.IntText(value=0, layout=widgets.Layout(width='50px'))
osc_send_ip3_gui = widgets.IntText(value=1, layout=widgets.Layout(width='50px'))
osc_send_port_label_gui = widgets.Label(value="Port", layout=widgets.Layout(width='50px'))
osc_send_port_gui = widgets.IntText(value=9007, layout=widgets.Layout(width='100px'))

osc_send_ip_gui = widgets.HBox([osc_send_ip_label_gui, osc_send_ip0_gui, osc_send_ip1_gui, osc_send_ip2_gui, osc_send_ip3_gui, osc_send_port_label_gui, osc_send_port_gui])
display(osc_send_ip_gui)

### Create Motion Player

In [4]:
motion_player.config = { 
    "file_name": mocap_file_gui.selected,
    "fps": mocap_fps_gui.value
    }

player = motion_player.MotionPlayer(motion_player.config)

node nr  0  name  Hips
node nr  1  name  RightUpLeg
node nr  2  name  RightLeg
node nr  3  name  RightFoot
node nr  4  name  RightToeBase
node nr  5  name  LeftUpLeg
node nr  6  name  LeftLeg
node nr  7  name  LeftFoot
node nr  8  name  LeftToeBase
node nr  9  name  Spine
node nr  10  name  Spine1
node nr  11  name  Spine2
node nr  12  name  Spine3
node nr  13  name  LeftShoulder
node nr  14  name  LeftArm
node nr  15  name  LeftForeArm
node nr  16  name  LeftHand
node nr  17  name  RightShoulder
node nr  18  name  RightArm
node nr  19  name  RightForeArm
node nr  20  name  RightHand
node nr  21  name  Neck
node nr  22  name  Head


### Create OSC Sender

In [5]:
motion_sender.config["ip"] = f"{osc_send_ip0_gui.value}.{osc_send_ip1_gui.value}.{osc_send_ip2_gui.value}.{osc_send_ip3_gui.value}"
motion_sender.config["port"] = osc_send_port_gui.value

osc_sender = motion_sender.OscSender(motion_sender.config)

### Create Motion GUI

In [6]:
motion_gui.config["player"] = player
motion_gui.config["sender"] = osc_sender

app = QtWidgets.QApplication(sys.argv)
gui = motion_gui.MotionGui(motion_gui.config)

# set close event
def closeEvent():
    QtWidgets.QApplication.quit()
app.lastWindowClosed.connect(closeEvent) # myExitHandler is a callable

### Create OSC Receiver

In [7]:
motion_control.config["gui"] = gui
motion_control.config["ip"] = f"{osc_rec_ip0_gui.value}.{osc_rec_ip1_gui.value}.{osc_rec_ip2_gui.value}.{osc_rec_ip3_gui.value}"
motion_control.config["port"] = osc_rec_port_gui.value

osc_control = motion_control.MotionControl(motion_control.config)

### Start Application

In [8]:
osc_control.start()
gui.show()
app.exec_()

0